In [67]:
%pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
df = pd.read_excel('data/healthcare_data.xlsx')
df.shape
df.columns

Index(['Patient_ID', 'Age', 'Gender', 'Blood_Type', 'Admission_Date',
       'Discharge_Date', 'Diagnosis', 'Treatment', 'Medication',
       'Treatment_Cost', 'Insurance_Provider', 'Length_of_Stay',
       'Readmission_Status', 'Doctor_Specialty'],
      dtype='object')

In [5]:
df.isna().count()

Patient_ID            10000
Age                   10000
Gender                10000
Blood_Type            10000
Admission_Date        10000
Discharge_Date        10000
Diagnosis             10000
Treatment             10000
Medication            10000
Treatment_Cost        10000
Insurance_Provider    10000
Length_of_Stay        10000
Readmission_Status    10000
Doctor_Specialty      10000
dtype: int64

In [6]:
df['Diagnosis'].value_counts()

Diagnosis
Hypertension               1985
Stroke                     1524
Diabetes                   1468
COPD                       1045
Pneumonia                  1039
Asthma                     1015
Sepsis                      990
Coronary Artery Disease     934
Name: count, dtype: int64

In [65]:
import plotly.express as px

# Prepare data
df_counts = df["Diagnosis"].value_counts().reset_index()
df_counts.columns = ["Diagnosis", "Count"]

# Define colors (Top Diagnosis in Navy Blue, Others in Gray)
colors = ["navy"] + ["teal"] * (len(df_counts) - 1)

# Create an interactive horizontal bar chart
fig = px.bar(df_counts, 
             x="Count", 
             y="Diagnosis", 
             text="Count", 
             orientation='h', 
             color=df_counts["Diagnosis"],
             color_discrete_sequence=colors)

# Customize layout
fig.update_traces(textposition="auto", textfont_size=14)
fig.update_layout(title="Top Diagnoses",
                  xaxis=dict(showticklabels=False),
                  yaxis=dict(title="Diagnosis"), 
                  showlegend=False,
                  template="plotly_white")

# Show the plot
fig.show()


In [26]:
df.head()

,Patient_ID,Age,Gender,Blood_Type,Admission_Date,Discharge_Date,Diagnosis,Treatment,Medication,Treatment_Cost,Insurance_Provider,Length_of_Stay,Readmission_Status,Doctor_Specialty,Age_Group
0,PID_00001,58,Male,A-,2024-06-08,2024-06-22,Coronary Artery Disease,Physical Therapy,Aspirin,3862.34,Cigna,14,Yes,General Surgery,55-59
1,PID_00002,56,Male,AB-,2024-10-24,2024-11-15,Coronary Artery Disease,Medication,Insulin,23422.26,BlueCross,22,Yes,Oncology,55-59
2,PID_00003,47,Female,O-,2024-04-04,2024-04-19,Sepsis,Physical Therapy,Insulin,37826.58,BlueCross,15,No,General Surgery,45-49
3,PID_00004,77,Male,AB+,2023-09-02,2023-09-30,Coronary Artery Disease,Medication,Aspirin,8273.69,Cigna,28,No,General Surgery,75-79
4,PID_00005,52,Male,AB-,2024-03-05,2024-03-31,Hypertension,Surgery,Albuterol,1560.80,UnitedHealth,26,No,Cardiology,50-54


In [64]:

# Count gender occurrences
df_gender = df["Gender"].value_counts().reset_index()
df_gender.columns = ["Gender", "Count"]

# Create bar plot
fig = px.bar(df_gender, 
             x="Gender", 
             y="Count", 
             text="Count", 
             color="Gender", 
             color_discrete_sequence=["navy", "teal"],  # Customize colors
             title="Gender Distribution")

# Customize layout
fig.update_traces(textposition="inside", textfont_size=14)
fig.update_layout(
    xaxis_title="Gender", 
    yaxis_title="",  # Remove axis title
    yaxis=dict(showgrid=False, showticklabels=False),  # Hide gridlines and scale
    template="plotly_white",
    showlegend=False
)

# Show plot
fig.show()


In [63]:

# Count occurrences of each diagnosis by gender
df_diag_gender = df.groupby(["Diagnosis", "Gender"]).size().reset_index(name="Count")

# Create grouped bar plot
fig = px.bar(df_diag_gender, 
             x="Diagnosis", 
             y="Count", 
             color="Gender", 
             barmode="group",  # Groups bars by gender
             text="Count",
             title="Diagnosis Distribution by Gender",
             color_discrete_map={"Male": "navy", "Female": "teal","Other":"gray"})  # Custom gender colors

# Customize layout
fig.update_traces(textposition="outside", textfont_size=10)
fig.update_layout(
    xaxis_title="Diagnosis", 
    yaxis_title="",  # Remove y-axis title
    yaxis=dict(showgrid=False, showticklabels=False),  # Hide gridlines and scale
    xaxis=dict(tickangle=-45),  # Rotate x-axis labels for better readability
    template="plotly_white"
)

# Show plot
fig.show()
         

In [20]:



# Bin ages into 5-year intervals
bins = list(range(0, df["Age"].max() + 5, 5))  
labels = [f"{b}-{b+4}" for b in bins[:-1]]  

df["Age_Group"] = pd.cut(df["Age"], bins=bins, labels=labels, right=False)

# Count occurrences of each diagnosis by age group
df_age_diag = df.groupby(["Age_Group"]).size().reset_index(name="Count")

# Sort age groups based on total diagnosis count (descending order)
df_age_diag = df_age_diag.sort_values(by="Count", ascending=False)

# Identify the top 3 age groups
top_3_groups = df_age_diag["Age_Group"].head(3).tolist()

# Assign colors: Maroon for top 3, Gray for others
df_age_diag["Color"] = df_age_diag["Age_Group"].apply(lambda x: "maroon" if x in top_3_groups else "gray")

# Create bar chart
fig = px.bar(df_age_diag, 
             x="Age_Group", 
             y="Count", 
             text="Count",
             title="Age Groups with Highest Diagnoses",
             color="Color",
             color_discrete_map={"maroon": "navy", "gray": "gray"})  

# Add a **VERTICAL** dotted line separating the top 3 groups
fig.add_shape(
    type="line",
    x0=2.5, x1=2.5,  # Position at the boundary of the top 3
    y0=0, y1=df_age_diag["Count"].max() * 1.1,  
    line=dict(color="red", width=1, dash="dot")
)

# Customize layout
fig.update_traces(textposition="outside", textfont_size=12)
fig.update_layout(
    xaxis_title="Age Group (5-year bins)", 
    yaxis_title="Total Diagnoses",
    xaxis=dict(type="category"),  
    template="plotly_white",
    showlegend=False  
)

# Show plot
fig.show()


C:\Users\Nensok Obadiah\AppData\Local\Temp\ipykernel_9756\3733160753.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [11]:

# Bin ages into 5-year intervals
bins = list(range(0, df["Age"].max() + 5, 5))  # Create bins from 0 to max age in steps of 5
labels = [f"{b}-{b+4}" for b in bins[:-1]]  # Label bins (e.g., "0-4", "5-9")

df["Age_Group"] = pd.cut(df["Age"], bins=bins, labels=labels, right=False)

# Count occurrences of each diagnosis by age group
df_age_diag = df.groupby(["Age_Group", "Diagnosis"]).size().reset_index(name="Count")

# **Remove age bins with no records**
df_age_diag = df_age_diag[df_age_diag["Count"] > 0]

# Create an interactive bar chart
fig = px.bar(df_age_diag, 
             x="Age_Group", 
             y="Count", 
             color="Diagnosis",
             text="Count",
             title="Diagnoses by Age Group",
             barmode="group",  # Group bars by diagnosis for comparison
             color_discrete_sequence=px.colors.qualitative.Set1)  # Distinct colors

# Customize layout
fig.update_traces(textposition="outside", textfont_size=12)
fig.update_layout(
    xaxis_title="Age Group", 
    yaxis_title="Total Diagnoses",
    xaxis=dict(type="category"),  # Ensure proper categorical sorting
    template="plotly_white",
    showlegend=True,
    legend_title="Diagnosis"
)

# Show plot
fig.show()


C:\Users\Nensok Obadiah\AppData\Local\Temp\ipykernel_9756\3072692700.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [62]:
import plotly.express as px
import plotly.graph_objects as go

# Count the number of doctors per specialty
df_specialty = df["Doctor_Specialty"].value_counts().reset_index()
df_specialty.columns = ["Specialty", "Count"]

# Identify the specialty with the highest count
max_specialty = df_specialty.iloc[0]  # Get top specialty
highlight_color = "navy"  # Color for the highest specialty

# Create a horizontal bar chart
fig = px.bar(df_specialty, 
             y="Specialty", 
             x="Count", 
             text="Count",
             title="Doctor Specialties",
             orientation="h",
             color=df_specialty["Specialty"].apply(lambda x: highlight_color if x == max_specialty["Specialty"] else "teal"),
             color_discrete_map="identity")  # Keep custom colors

# Customize layout (remove horizontal scale)
fig.update_traces(textposition="outside", textfont_size=12)
fig.update_layout(
    xaxis_title="",
    yaxis_title="Doctor Specialty",
    xaxis=dict(
        showgrid=False,  
        showticklabels=False,  
        zeroline=False  
    ),
    yaxis=dict(categoryorder="total ascending"),
    template="plotly_white",
    showlegend=False
)

# Show plot
fig.show()


In [61]:

# Aggregate total treatment cost per treatment type
df_treatment_cost = df.groupby("Treatment")["Treatment_Cost"].sum().reset_index()
df_treatment_cost = df_treatment_cost.sort_values(by="Treatment_Cost", ascending=False)

# Identify the treatment with the highest cost
max_treatment = df_treatment_cost.iloc[0]  # Get highest cost treatment
highlight_color = "navy"  # Color for highest treatment
other_color = "teal"  # Color for others

# Convert Treatment_Cost to currency format with ₦ symbol
df_treatment_cost["Formatted_Cost"] = df_treatment_cost["Treatment_Cost"].apply(lambda x: f"₦{x:,.2f}")

# Create a horizontal bar chart
fig = px.bar(df_treatment_cost, 
             y="Treatment", 
             x="Treatment_Cost", 
             text=df_treatment_cost["Formatted_Cost"],  # Use formatted cost
             title="Total Treatment Cost by Treatment",
             orientation="h",
             color=df_treatment_cost["Treatment"].apply(lambda x: highlight_color if x == max_treatment["Treatment"] else other_color),
             color_discrete_map="identity")  # Keep custom colors

# Remove horizontal scale
fig.update_layout(
    xaxis_title="",
    yaxis_title="Treatment",
    xaxis=dict(
        showgrid=False,   # Remove gridlines
        showticklabels=False,  # Hide scale labels
        zeroline=False,  # Remove zero line
    ),
    yaxis=dict(categoryorder="total ascending"),  # Order by cost
    template="plotly_white",
    showlegend=False
)

# Add annotation for highest cost treatment
fig.add_annotation(
    x=max_treatment["Treatment_Cost"] + (max_treatment["Treatment_Cost"] * 0.05),
    y=df_treatment_cost[df_treatment_cost["Treatment"] == max_treatment["Treatment"]].index[0],
    text=f"Highest Cost: {max_treatment['Treatment']} (₦{max_treatment['Treatment_Cost']:,.2f})",
    showarrow=False,
    font=dict(size=12, color="red"),
    
)

# Show plot
fig.show()


In [43]:


# Ensure Admission_Date and Discharge_Date are in datetime format
df["Admission_Date"] = pd.to_datetime(df["Admission_Date"])
df["Discharge_Date"] = pd.to_datetime(df["Discharge_Date"])

# Compute the number of days admitted
df["Days_Admitted"] = (df["Discharge_Date"] - df["Admission_Date"]).dt.days

# Compute the average admission days per diagnosis
df_avg_admission = df.groupby("Diagnosis")["Days_Admitted"].mean().reset_index()
df_avg_admission.columns = ["Diagnosis", "Avg_Days_Admitted"]

# Sort by average admission days in descending order
df_avg_admission = df_avg_admission.sort_values(by="Avg_Days_Admitted", ascending=False)

# Create a horizontal bar chart
fig = px.bar(df_avg_admission, 
             x="Avg_Days_Admitted", 
             y="Diagnosis", 
             text=df_avg_admission["Avg_Days_Admitted"].round(1),  # Show rounded values
             title="Average Admission Days by Diagnosis",
             orientation="h",
             color_discrete_sequence=["navy"])  # Custom color

# Customize layout
fig.update_traces(textposition="inside")
fig.update_layout(
    xaxis_title="Average Days Admitted",
    yaxis_title="Diagnosis",
    yaxis=dict(categoryorder="total ascending"),  # Order by highest avg days
    template="plotly_white",
    showlegend=False
)

# Show plot
fig.show()


In [60]:


# Convert Admission_Date to datetime if not already
df["Admission_Date"] = pd.to_datetime(df["Admission_Date"])

# Group by month and count admissions
admissions_per_month = df.groupby(df["Admission_Date"].dt.to_period("M")).size().reset_index(name="Admissions")

# Convert Period to string for Plotly compatibility
admissions_per_month["Admission_Date"] = admissions_per_month["Admission_Date"].astype(str)

# Create smooth line chart with markers
fig1 = px.line(admissions_per_month, 
               x="Admission_Date", 
               y="Admissions", 
               title="Monthly Patient Admissions Trend", 
               labels={"Admission_Date": "Month", "Admissions": "Number of Admissions"},
               markers=True)  # Adds markers at data points

# Smooth the lines by reducing line shape jaggedness
fig1.update_traces(line_shape="spline", line=dict(width=3))

# Show the plot
fig1.show()



In [57]:
cost_per_diagnosis = df.groupby("Diagnosis")["Treatment_Cost"].sum().reset_index().sort_values(by="Treatment_Cost", ascending=False)
fig2 = px.bar(cost_per_diagnosis, x="Treatment_Cost", y="Diagnosis", orientation="h", title="Total Treatment Cost per Diagnosis", text=cost_per_diagnosis["Treatment_Cost"].apply(lambda x: f"₦{x:,.2f}"))
fig2.update_traces(textposition="outside", marker_color="teal")
fig2.show()

In [56]:


# Count occurrences of each diagnosis
diagnosis_counts = df["Diagnosis"].value_counts().reset_index()
diagnosis_counts.columns = ["Diagnosis", "Count"]

# Create a bar chart (histogram-like)
fig4 = px.bar(diagnosis_counts, 
              x="Diagnosis", 
              y="Count", 
              title="Disease Burden Distribution",
              labels={"Diagnosis": "Disease", "Count": "Number of Cases"},
              text="Count")  # Show counts on bars

# Improve readability
fig4.update_traces(marker_color="teal", textposition="outside")

# Rotate x-axis labels if needed
fig4.update_layout(xaxis_tickangle=-45, template="plotly_white")

# Show the plot
fig4.show()
